# Install and import packages
--------

In [1]:
# install/import quantum gym environments
#!pip install git+https://github.com/qdevpsi3/quantum-arch-search.git

# install/import stable baselines 3
#!pip install stable_baselines3

In [2]:
import gym
import numpy as np
import torch.optim as optim
from stable_baselines3 import A2C, PPO
from stable_baselines3.common.evaluation import evaluate_policy
import qas_gym
# from scipy.stats import unitary_group


# Basic Environment
------
Create your gym environment :

In [3]:
from qulacs import QuantumState
from scipy.linalg import expm

# x = unitary_group.rvs(2**2)
# initial = np.dot(x , np.asarray([1, 0,0,0]))
# print(initial)
n = 3
state = QuantumState(n)
state.set_Haar_random_state(seed=1)
initial = state.get_vector()
print(initial)
# define Hamiltonian for evolution
from qulacs import Observable
observable = Observable(n)
observable.add_operator(1.0, "X 0")
observable.add_operator(1.0, "Z 1")
observable.add_operator(0.5, "X 0 X 1")
observable.add_operator(0.5, "X 1 X 2")
# ref to https://pennylane.ai/qml/demos/tutorial_qaoa_intro/

obs = observable.get_matrix()
# target = QuantumState(n)
dt = 0.2
target = expm(1j *obs.todense() * dt) @ initial 
print(target)

# # target = np.asarray([0.70710678+0.j,0. +0.j,0. +0.j, 0.70710678+0.j])


[ 0.57833664-0.17703067j -0.17428632-0.04541808j  0.40305753-0.07842465j
  0.17057319+0.40309603j  0.1880943 +0.02558741j  0.02670883+0.21334971j
  0.13163303-0.23130147j  0.15675915-0.25277309j]
[ 0.5722299 -0.05463616j -0.1208126 +0.09180177j  0.28526329-0.10110261j
  0.2659538 +0.48572908j  0.15920145+0.10774843j -0.06176448+0.28068538j
  0.12925355-0.17088505j  0.13672501-0.25323441j]


In [4]:
def calculate_fidelity(initial, target):
    inner = np.inner(np.conj(initial), target)
    return np.conj(inner) * inner

fidelity = calculate_fidelity(initial,target)
print(fidelity.real)

0.9130362399873229


Display the action gates : 

In [5]:
# for idx, gate in enumerate(env.action_gates):
#     print('Action({:02d}) --> {}'.format(idx, gate))

Diplay the state observables : 

In [6]:
# for idx, observable in enumerate(env.state_observables):
#     print('State({:02d}) --> {}'.format(idx, observable))

# for idx, observable in enumerate(env.pauli_observables):
#     print('State({:02d}) --> {}'.format(idx, observable))   

In [7]:
# from cirq.work.observable_measurement import measure_observables, RepetitionsStoppingCriteria
# import cirq
# # circuit = env._get_cirq(maybe_add_noise=True)
# # sample_pauli_obs = env.pauli_observables[0]
# circuit = cirq.Circuit(cirq.I(qubit) for qubit in env.qubits)
# # result = measure_observables(
# #                 circuit, env.pauli_observables, cirq.Simulator(), stopping_criteria=RepetitionsStoppingCriteria(100))
# K=10
# # sample_pauli_idx = np.random.choice(len(env.pauli_observables), K)
# sample_pauli = [env.pauli_observables[i] for i in np.random.choice(len(env.pauli_observables), K)]
# result = measure_observables(
#     circuit, sample_pauli, cirq.Simulator(), stopping_criteria=RepetitionsStoppingCriteria(100))
  
# # print(result[15].mean, result[15].variance)

In [8]:
# for observable in env.pauli_observables:
#    print(type(observable))
#    for qubit, pauli in observable.items():
#       print(qubit, pauli)

# print(result[1].mean, result[1].variance)

# A2C Agent
------

In [ ]:
# # Parameters
# gamma = 0.99
# learning_rate = 0.0001
# policy_kwargs = dict(optimizer_class=optim.Adam)

# # Agent
# a2c_model = A2C("MlpPolicy",
#                 env,
#                 gamma=gamma,
#                 learning_rate=learning_rate,
#                 policy_kwargs=policy_kwargs,
#                 tensorboard_log='logs/')

In [ ]:
# a2c_model.learn(total_timesteps=20000)

In [ ]:
# # 从初始态开始，每一步用训练好的agent对当前state选择添加的门（action）
# state = env.reset()
# print(state)
# done = False
# while not done:
#     action = a2c_model.predict(state)
#     state, reward, done, info = env.step(action[0])
#     #展示当前的线路 和 state
#     # env.render()
#     print(state)

# print(info['circuit'])
# print(info['fidelity'])

# PPO Model
------

In [9]:
# Environment
# Parameters 
env_name = 'BasicThreeQubit-v0' # 两比特环境-无噪声
fidelity_threshold = 0.95
reward_penalty = 0.01
max_timesteps = 5
env = gym.make(env_name, target = target,
               fidelity_threshold=fidelity_threshold,
               reward_penalty=reward_penalty,
               max_timesteps=max_timesteps,
               initial = initial)
               
# Parameters
gamma = 0.99
n_epochs = 4
clip_range = 0.2
learning_rate = 0.0001
policy_kwargs = dict(optimizer_class=optim.Adam)


# Agent
ppo_model = PPO("MlpPolicy",
                env,
                gamma=gamma,
                n_epochs=n_epochs,
                clip_range=clip_range,
                learning_rate=learning_rate,
                policy_kwargs=policy_kwargs,
                tensorboard_log='logs/')
ppo_model.learn(total_timesteps=20000)

C:\Users\Mac\AppData\Roaming\Python\Python311\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
C:\Users\Mac\AppData\Roaming\Python\Python311\site-packages\gym\utils\passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
C:\Users\Mac\AppData\Roaming\Python\Python311\site-packages\gym\utils\passive_env_checker.py:187: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
C:\Users\Mac\AppData\Roaming\Python\Python311\site-packages\

number of gates:1,  number of param gates:1
Fidelity: 0.917002413678215
number of gates:2,  number of param gates:1
Fidelity: 0.11338680654957495
number of gates:3,  number of param gates:2
Fidelity: 0.1301192479282621
number of gates:4,  number of param gates:2
Fidelity: 0.6232053957335364
number of gates:5,  number of param gates:2
Fidelity: 0.29525988943953296
number of gates:1,  number of param gates:0
Fidelity: 0.14020647134765776
number of gates:2,  number of param gates:0
Fidelity: 0.4451074607598381
number of gates:3,  number of param gates:0
Fidelity: 0.19074270940162802
number of gates:4,  number of param gates:1
Fidelity: 0.5396363911241906
number of gates:5,  number of param gates:1
Fidelity: 0.470074996218268
number of gates:1,  number of param gates:0
Fidelity: 0.14020647134765776
number of gates:2,  number of param gates:1
Fidelity: 0.23425728456560954
number of gates:3,  number of param gates:2
Fidelity: 0.7086590268147951
number of gates:4,  number of param gates:3
Fid

In [ ]:
# 从初始态开始，每一步用训练好的agent对当前state选择添加的门（action）
state = env.reset()
print(state)
done = False
while not done:
    action = ppo_model.predict(state)
    state, reward, done, info = env.step(action[0])
    #展示当前的线路 和 state
    # env.render()
    # print(state)
    print(info['fidelity'])
print("Final circuit: ")    
env.render()


[ 0.06793337  0.32231771  0.28256146  0.33609777 -0.2471595  -0.03896541
  0.1718993  -0.3541682   0.51687813]
number of gates:1,  number of param gates:1
Parameters:  [0.36633318]
0.9442380263481496
number of gates:2,  number of param gates:2
Parameters:  [ 0.36482484 -0.00470016]
0.9442413015142458
number of gates:3,  number of param gates:3
Parameters:  [ 0.32128067 -0.03986639  0.0725502 ]
0.944909645425868
number of gates:4,  number of param gates:4
Parameters:  [ 0.32887958 -0.01538217  0.03475079 -0.12271428]
0.9474876471152409
number of gates:5,  number of param gates:5
Parameters:  [-0.74686981 -0.04710462 -0.01941937 -0.26296351  1.10489054]
0.9596343964322194
number of gates:6,  number of param gates:5
Parameters:  [-0.89251569 -0.26247327  0.63310656  2.59918084  3.11904881]
0.7374227765675543
number of gates:7,  number of param gates:6
Parameters:  [-0.89266704 -0.26240144  0.6330436   2.59909103  2.85597301  0.26319455]
0.7374227710307236
Final circuit: 

0: ───I───Rx(-0.

C:\Users\Mac\AppData\Roaming\Python\Python311\site-packages\gym\utils\passive_env_checker.py:272: UserWarning: WARN: No render modes was declared in the environment (env.metadata['render_modes'] is None or not defined), you may have trouble when calling `.render()`.
  logger.warn(


In [ ]:
# test effects of max depth on the result
max_timesteps = [1,3,5]
opt_fidelity = np.zeros(len(max_timesteps))
for i, mts in enumerate(max_timesteps):
    # Environment
    env = gym.make(env_name, target = target,
                fidelity_threshold=fidelity_threshold,
                reward_penalty=reward_penalty,
                max_timesteps=mts,
                initial = initial)

    ppo_model = PPO("MlpPolicy",
                    env,
                    gamma=gamma,
                    n_epochs=n_epochs,
                    clip_range=clip_range,
                    learning_rate=learning_rate,
                    policy_kwargs=policy_kwargs,
                    tensorboard_log='logs/')

    ppo_model.learn(total_timesteps=20000)

   # simulate to get fidelity
    state = env.reset()
    print(state)
    done = False
    while not done:
        action = ppo_model.predict(state)
        state, reward, done, info = env.step(action[0])
        #展示当前的线路 和 state
        env.render()
        # print(state)
        print(info['fidelity'])

    opt_fidelity[i] = info['fidelity']
    


C:\Users\Mac\AppData\Roaming\Python\Python311\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
C:\Users\Mac\AppData\Roaming\Python\Python311\site-packages\gym\utils\passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
C:\Users\Mac\AppData\Roaming\Python\Python311\site-packages\gym\utils\passive_env_checker.py:187: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
C:\Users\Mac\AppData\Roaming\Python\Python311\site-packages\

number of gates:1,  number of param gates:1
Parameters:  [-0.15873686]
number of gates:1,  number of param gates:0
Parameters:  None
number of gates:1,  number of param gates:1
Parameters:  [0.36628623]
number of gates:1,  number of param gates:1
Parameters:  [0.20992336]
number of gates:1,  number of param gates:1
Parameters:  [0.15827115]
number of gates:1,  number of param gates:0
Parameters:  None
number of gates:1,  number of param gates:1
Parameters:  [0.13923181]
number of gates:1,  number of param gates:1
Parameters:  [0.26743129]
number of gates:1,  number of param gates:1
Parameters:  [0.15819395]
number of gates:1,  number of param gates:1
Parameters:  [0.00656106]
number of gates:1,  number of param gates:0
Parameters:  None
number of gates:1,  number of param gates:0
Parameters:  None
number of gates:1,  number of param gates:0
Parameters:  None
number of gates:1,  number of param gates:0
Parameters:  None
number of gates:1,  number of param gates:0
Parameters:  None
numbe

KeyboardInterrupt: 

In [ ]:
opt_fidelity

array([0.98761793, 0.98761793, 0.9854617 ])

# Results
------

In [ ]:
help(gym.envs.register)

Help on function register in module gym.envs.registration:

register(id: str, entry_point: Union[Callable, str], reward_threshold: Optional[float] = None, nondeterministic: bool = False, max_episode_steps: Optional[int] = None, order_enforce: bool = True, autoreset: bool = False, disable_env_checker: bool = False, apply_api_compatibility: bool = False, **kwargs)
    Register an environment with gym.
    
    The `id` parameter corresponds to the name of the environment, with the syntax as follows:
    `(namespace)/(env_name)-v(version)` where `namespace` is optional.
    
    It takes arbitrary keyword arguments, which are passed to the `EnvSpec` constructor.
    
    Args:
        id: The environment id
        entry_point: The entry point for creating the environment
        reward_threshold: The reward threshold considered to have learnt an environment
        nondeterministic: If the environment is nondeterministic (even with knowledge of the initial seed and all actions)
        m

In [ ]:
# 从初始态开始，每一步用训练好的agent对当前state选择添加的门（action）
state = env.reset()
done = False
while not done:
    action = ppo_model.predict(state)
    state, reward, done, info = env.step(action[0])
    #展示当前的线路 和 state
    env.render()
    print(state)


[ 0.42507896+0.14228169j  0.30719836+0.10905986j -0.25625132-0.63755635j
  0.30179013-0.35997053j]
number of gates:1,  number of param gates:1
Parameters:  [-0.02877633]

0: ───I─────────────────

1: ───I───Ry(-0.009π)───
[ 0.59653632  0.57460298  0.34615809  0.14956316 -0.75609493 -0.46021832]


In [14]:
#import cirq
from qulacs import ParametricQuantumCircuit
ansatz = ParametricQuantumCircuit(4)


In [15]:
help(ansatz.add_parametric_multi_Pauli_rotation_gate)

Help on method add_parametric_multi_Pauli_rotation_gate in module qulacs_core:

add_parametric_multi_Pauli_rotation_gate(...) method of qulacs_core.ParametricQuantumCircuit instance
    add_parametric_multi_Pauli_rotation_gate(self: qulacs_core.ParametricQuantumCircuit, index_list: List[int], pauli_ids: List[int], angle: float) -> None
    
    Add parametric multi-qubit Pauli rotation gate



In [2]:
from qulacs.gate import PauliRotation
target_list = [0,3,5]
pauli_index = [1,3,1] # 1:X , 2:Y, 3:Z
angle = 0.5
gate = PauliRotation(target_list, pauli_index, angle) # = exp(i angle/2 X_0 Z_3 X_5)
gate.get_target_index_list

'Pauli-rotation'

In [12]:
import json
a = json.loads(gate.to_json())

In [13]:
a.keys()

dict_keys(['name', 'angle', 'pauli'])

In [18]:
[int(_tmp['pauli_id']) for _tmp in a['pauli']['pauli_list']]

[1, 3, 1]

In [55]:
import cirq
a = cirq.XX()
help(a)

ValueError: Wrong number of qubits for <cirq.XX>. Expected 2 qubits but got <[]>.

In [44]:
a = cirq.XXPowGate()
qubits = cirq.LineQubit.range(3)

ValueError: Wrong number of qubits for <cirq.XX>. Expected 2 qubits but got <[cirq.LineQubit(1)]>.

In [54]:
import cirq
# help(cirq.SWAP)
help(cirq.XX)


Help on XXPowGate in module cirq.ops.parity_gates:

cirq.XX
    The tensor product of two X gates.
    
    Useful for creating `cirq.XXPowGate`s via `cirq.XX**t`.
    
    This is the `exponent=1` instance of `cirq.XXPowGate`.

